**<h1 align="center">Image Segmentation</h1>**

## Script from HuggingFace (do not use)

In [3]:
import io
import requests
from PIL import Image
import torch
import numpy

from transformers import DetrFeatureExtractor, DetrForSegmentation
from transformers.models.detr.feature_extraction_detr import rgb_to_id

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

# prepare image for the model
inputs = feature_extractor(images=image, return_tensors="pt")

# forward pass
outputs = model(**inputs)

# use the `post_process_panoptic` method of `DetrFeatureExtractor` to convert to COCO format
processed_sizes = torch.as_tensor(inputs["pixel_values"].shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes)[0]

# the segmentation is stored in a special-format png
panoptic_seg = Image.open(io.BytesIO(result["png_string"]))
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8)
# retrieve the ids corresponding to each mask
panoptic_seg_id = rgb_to_id(panoptic_seg)

c:\Users\Tony\Documents\GitHub Repos\ImageSegmentationTest\.venv\Lib\site-packages\transformers\models\detr\feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/detr-resnet-50-panoptic were not used when initializing DetrForSegmentation: ['detr.model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

## Model Download

In [6]:
from transformers import DetrForSegmentation, DetrImageProcessor

# Define a local directory to store the model
local_model_dir = "./detr-resnet-50-panoptic"

# Download and save the model & processor locally
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")
model.save_pretrained(local_model_dir)

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50-panoptic")
processor.save_pretrained(local_model_dir)

print(f"Model and processor saved to: {local_model_dir}")

Some weights of the model checkpoint at facebook/detr-resnet-50-panoptic were not used when initializing DetrForSegmentation: ['detr.model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model and processor saved to: ./detr-resnet-50-panoptic


## Load Model

In [ ]:
feature_extractor = DetrImageProcessor.from_pretrained(local_model_dir)
model = DetrForSegmentation.from_pretrained(local_model_dir)

print("Model loaded from local directory!")

Model loaded from local directory!


## Check where model is stored (from HuggingFace cache)

In [10]:
from huggingface_hub import hf_hub_download
import os

# Get the default cache directory
cache_dir = os.path.expanduser("~/.cache/huggingface/hub")

print("Hugging Face model cache directory:", cache_dir)

Hugging Face model cache directory: C:\Users\Tony/.cache/huggingface/hub
